In [127]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.api as sm
from sklearn import metrics 
from xgboost import XGBClassifier


In [128]:
df = pd.read_csv('E:/Projects/AV_Hackathon/train_ZoGVYWq.csv')
df.shape

(79853, 13)

### Exploratory Data Analysis

In [129]:
# Renewal perc
(df['renewal'].value_counts())/df.shape[0]

1    0.93741
0    0.06259
Name: renewal, dtype: float64

In [130]:
df.dtypes

id                                    int64
perc_premium_paid_by_cash_credit    float64
age_in_days                           int64
Income                                int64
Count_3-6_months_late               float64
Count_6-12_months_late              float64
Count_more_than_12_months_late      float64
application_underwriting_score      float64
no_of_premiums_paid                   int64
sourcing_channel                     object
residence_area_type                  object
premium                               int64
renewal                               int64
dtype: object

In [131]:
df.isnull().sum()

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 97
Count_6-12_months_late                97
Count_more_than_12_months_late        97
application_underwriting_score      2974
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
renewal                                0
dtype: int64

In [132]:
df.describe()

,id,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,premium,renewal
count,79853.000000,79853.000000,79853.000000,7.985300e+04,79756.000000,79756.000000,79756.000000,76879.000000,79853.000000,79853.000000,79853.000000
mean,57167.166368,0.314288,18846.696906,2.088472e+05,0.248671,0.078188,0.060008,99.067291,10.863887,10924.507533,0.937410
std,32928.970160,0.334915,5208.719136,4.965826e+05,0.691468,0.436507,0.312023,0.739799,5.170687,9401.676542,0.242226
min,2.000000,0.000000,7670.000000,2.403000e+04,0.000000,0.000000,0.000000,91.900000,2.000000,1200.000000,0.000000
25%,28640.000000,0.034000,14974.000000,1.080100e+05,0.000000,0.000000,0.000000,98.810000,7.000000,5400.000000,1.000000
50%,57262.000000,0.167000,18625.000000,1.665600e+05,0.000000,0.000000,0.000000,99.210000,10.000000,7500.000000,1.000000
75%,85632.000000,0.538000,22636.000000,2.520900e+05,0.000000,0.000000,0.000000,99.540000,14.000000,13800.000000,1.000000
max,114076.000000,1.000000,37602.000000,9.026260e+07,13.000000,17.000000,11.000000,99.890000,60.000000,60000.000000,1.000000


In [133]:
# Perc_premium - Policies renewed have less perc paid by cash or credit
print(df['perc_premium_paid_by_cash_credit'].quantile([0.25,0.35,0.5,0.75,0.8,0.95,0.99]))
print(df.groupby('renewal').aggregate({'perc_premium_paid_by_cash_credit' : 'mean'}))
print(df.groupby('renewal').aggregate({'perc_premium_paid_by_cash_credit' : 'median'}))
print(min(df.perc_premium_paid_by_cash_credit))
print(max(df.perc_premium_paid_by_cash_credit))

0.25    0.034
0.35    0.067
0.50    0.167
0.75    0.538
0.80    0.658
0.95    1.000
0.99    1.000
Name: perc_premium_paid_by_cash_credit, dtype: float64
         perc_premium_paid_by_cash_credit
renewal                                  
0                                0.626626
1                                0.293433
         perc_premium_paid_by_cash_credit
renewal                                  
0                                   0.728
1                                   0.148
0.0
1.0


In [134]:
# Age of policy holders who renewed is slightly higher
print(df['age_in_days'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99]))
print(df.groupby('renewal').aggregate({'age_in_days' : 'mean'}))
print(df.groupby('renewal').aggregate({'age_in_days' : 'median'}))

0.10    12051.0
0.25    14974.0
0.35    16438.0
0.50    18625.0
0.75    22636.0
0.80    23368.0
0.95    27754.0
0.99    31396.0
Name: age_in_days, dtype: float64
          age_in_days
renewal              
0        16929.642657
1        18974.696867
         age_in_days
renewal             
0            16800.5
1            18630.0


In [135]:
# Adding a column - age_in_years
df['age_in_years'] = df['age_in_days']/365.25
print(df['age_in_years'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99,0.999]))
print(df.groupby('renewal').aggregate({'age_in_years' : 'mean'}))
print(df.groupby('renewal').aggregate({'age_in_years' : 'median'}))
print(min(df.age_in_years))
print(max(df.age_in_years))

0.100    32.993840
0.250    40.996578
0.350    45.004791
0.500    50.992471
0.750    61.973990
0.800    63.978097
0.950    75.986311
0.990    85.957563
0.999    91.975765
Name: age_in_years, dtype: float64
         age_in_years
renewal              
0           46.350835
1           51.949889
         age_in_years
renewal              
0           45.997262
1           51.006160
20.9993155373
102.948665298


In [136]:
print(df['Income'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99,0.998]))
print(df.groupby('renewal').aggregate({'Income' : 'mean'}))
print(df.groupby('renewal').aggregate({'Income' : 'median'}))
print(min(df.Income))
print(max(df.Income))

0.100      71200.00
0.250     108010.00
0.350     130100.00
0.500     166560.00
0.750     252090.00
0.800     279030.00
0.950     450050.00
0.990     771078.00
0.998    1826578.88
Name: Income, dtype: float64
                Income
renewal               
0        177058.799520
1        210969.651727
         Income
renewal        
0        138100
1        168870
24030
90262600


In [137]:
print(df.groupby('renewal').aggregate({'Count_3-6_months_late' : 'sum','Count_6-12_months_late' : 'sum','Count_more_than_12_months_late' : 'sum'}))

         Count_3-6_months_late  Count_6-12_months_late  \
renewal                                                  
0                       4536.0                  2787.0   
1                      15297.0                  3449.0   

         Count_more_than_12_months_late  
renewal                                  
0                                1738.0  
1                                3048.0  


In [138]:
# No difference in no_of_premiums paid among the customers
print(df['no_of_premiums_paid'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99,0.999]))
print(df.groupby('renewal').aggregate({'no_of_premiums_paid' : 'mean'}))
print(df.groupby('renewal').aggregate({'no_of_premiums_paid' : 'median'}))
print(min(df.no_of_premiums_paid))
print(max(df.no_of_premiums_paid))

0.100     5.0
0.250     7.0
0.350     8.0
0.500    10.0
0.750    14.0
0.800    15.0
0.950    20.0
0.990    27.0
0.999    37.0
Name: no_of_premiums_paid, dtype: float64
         no_of_premiums_paid
renewal                     
0                  10.410364
1                  10.894169
         no_of_premiums_paid
renewal                     
0                         10
1                         10
2
60


In [139]:
# No difference in application_underwriting_score among the customers
print(df['application_underwriting_score'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99,0.999]))
print(df.groupby('renewal').aggregate({'application_underwriting_score' : 'mean'}))
print(df.groupby('renewal').aggregate({'application_underwriting_score' : 'median'}))
print(min(df.application_underwriting_score))
print(max(df.application_underwriting_score))


0.100    98.29
0.250    98.81
0.350    99.00
0.500    99.21
0.750    99.54
0.800    99.61
0.950    99.87
0.990    99.89
0.999    99.89
Name: application_underwriting_score, dtype: float64
         application_underwriting_score
renewal                                
0                             98.870256
1                             99.080407
         application_underwriting_score
renewal                                
0                                 99.05
1                                 99.22
91.9
99.89


In [140]:
# Not much difference in premium
print(df['premium'].quantile([0.1,0.25,0.35,0.5,0.75,0.8,0.95,0.99,0.999]))
print(df.groupby('renewal').aggregate({'premium' : 'mean'}))
print(df.groupby('renewal').aggregate({'premium' : 'median'}))
print(min(df.premium))
print(max(df.premium))

0.100     3300.0
0.250     5400.0
0.350     5700.0
0.500     7500.0
0.750    13800.0
0.800    15900.0
0.950    28500.0
0.990    51600.0
0.999    60000.0
Name: premium, dtype: float64
              premium
renewal              
0         9599.519808
1        11012.975753
         premium
renewal         
0           7500
1           7500
1200
60000


In [141]:
# Sourcing Channel A is the best for renewal
df.groupby(['renewal','sourcing_channel'])['id'].count()/df.groupby('renewal')['id'].count()

renewal  sourcing_channel
0        A                   0.469988
         B                   0.213285
         C                   0.180672
         D                   0.126851
         E                   0.009204
1        A                   0.544853
         B                   0.206346
         C                   0.148768
         D                   0.092512
         E                   0.007521
Name: id, dtype: float64

In [142]:
#'residence_area_type' is not much different among renewal/non-renewal customers  
df.groupby(['renewal','residence_area_type'])['id'].count()/df.groupby('renewal')['id'].count()

renewal  residence_area_type
0        Rural                  0.399760
         Urban                  0.600240
1        Rural                  0.396393
         Urban                  0.603607
Name: id, dtype: float64

### Data Treatment

In [143]:
# One-Hot Encoding of categorical columns
df2 = df.copy(deep = True)
cat = pd.get_dummies(df2[['residence_area_type','sourcing_channel']])
df2 = pd.concat([df2[df2.columns.difference(['residence_area_type','sourcing_channel'])],cat[cat.columns.difference([
'residence_area_type_Rural','sourcing_channel_E'])]],axis = 1)

In [144]:
df2.set_index('id',inplace = True)

In [145]:
# Mean Imputation
df2.loc[df2['Count_3-6_months_late'].isnull(),'Count_3-6_months_late'] = df2['Count_3-6_months_late'].mean()
df2.loc[df2['Count_6-12_months_late'].isnull(),'Count_6-12_months_late'] = df2['Count_6-12_months_late'].mean()
df2.loc[df2['Count_more_than_12_months_late'].isnull(),'Count_more_than_12_months_late'] = df2['Count_more_than_12_months_late'].mean()
df2.loc[df2['application_underwriting_score'].isnull(),'application_underwriting_score'] = df2['application_underwriting_score'].mean()


In [146]:
df2['late_payment'] = df2['Count_3-6_months_late'] + df2['Count_6-12_months_late'] + df2['Count_more_than_12_months_late']

In [147]:
df2['cash_payment'] = df2['perc_premium_paid_by_cash_credit'] * df2['premium']

In [148]:
df2['correct_payment'] = df2['no_of_premiums_paid'] - df2['late_payment']

### XgBoost Classifier with Cross Validation

In [149]:
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=10) # Define the split - into 2 folds 
model_xgb = XGBClassifier(learning_rate=0.1)
X = df2[df2.columns.difference(['renewal'])]
Y = df2.renewal
res=[]
for train_index, test_index in kf.split(X):
    X_train_cv, X_test_cv = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train_cv, y_test_cv = Y[Y.index.isin(train_index)], Y[Y.index.isin(test_index)]
    pred_cv_xgb = model_xgb.fit(X_train_cv,y_train_cv).predict_proba(X_test_cv)[:,1]
    res.append(metrics.roc_auc_score(y_test_cv,pred_cv_xgb))
print(res)
print(np.mean(res))

[0.83705305803167485, 0.84318321879978131, 0.83823098423225906, 0.82860124185195672, 0.86346281544967451, 0.84080803013105687, 0.85079558648890996, 0.85175150262960175, 0.83499075297804481, 0.84984554425902936]
0.843872273485


In [150]:
pd.DataFrame({'features': X_train_cv.columns,'importance' : model_xgb.feature_importances_})

,features,importance
0,Count_3-6_months_late,0.005882
1,Count_6-12_months_late,0.054412
2,Count_more_than_12_months_late,0.051471
3,Income,0.097059
4,age_in_days,0.135294
5,age_in_years,0.000000
6,application_underwriting_score,0.154412
7,cash_payment,0.044118
8,correct_payment,0.102941
9,late_payment,0.138235


### Test Data Treatment

In [151]:
df_test = pd.read_csv('E:/Projects/AV_Hackathon/test_66516Ee.csv')
df_test.shape

(34224, 12)

In [152]:
df_test.isnull().sum()

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 31
Count_6-12_months_late                31
Count_more_than_12_months_late        31
application_underwriting_score      1323
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
dtype: int64

In [153]:
df_test['age_in_years'] = df_test['age_in_days']/365.25

In [154]:
df_test.set_index('id',inplace = True)

In [155]:
df_test.describe()

,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,premium,age_in_years
count,34224.000000,34224.000000,3.422400e+04,34193.000000,34193.000000,34193.000000,32901.000000,34224.000000,34224.000000,34224.000000
mean,0.314457,18824.215346,2.028201e+05,0.238733,0.080718,0.058111,99.061898,10.890428,10834.721248,51.537893
std,0.334059,5246.525604,2.702536e+05,0.686162,0.454634,0.307046,0.742942,5.216867,9263.074506,14.364204
min,0.000000,7671.000000,2.403000e+04,0.000000,0.000000,0.000000,91.900000,2.000000,1200.000000,21.002053
25%,0.034000,14972.000000,1.063975e+05,0.000000,0.000000,0.000000,98.800000,7.000000,5400.000000,40.991102
50%,0.169000,18623.000000,1.650700e+05,0.000000,0.000000,0.000000,99.210000,10.000000,7500.000000,50.986995
75%,0.540000,22636.000000,2.500200e+05,0.000000,0.000000,0.000000,99.530000,14.000000,13800.000000,61.973990
max,1.000000,35785.000000,2.191455e+07,12.000000,10.000000,7.000000,99.890000,59.000000,60000.000000,97.973990


In [156]:
df_test.loc[df_test['Count_3-6_months_late'].isnull(),'Count_3-6_months_late'] = df_test['Count_3-6_months_late'].mean()
df_test.loc[df_test['Count_6-12_months_late'].isnull(),'Count_6-12_months_late'] = df_test['Count_6-12_months_late'].mean()
df_test.loc[df_test['Count_more_than_12_months_late'].isnull(),'Count_more_than_12_months_late'] = df_test['Count_more_than_12_months_late'].mean()
df_test.loc[df_test['application_underwriting_score'].isnull(),'application_underwriting_score'] = df_test['application_underwriting_score'].mean()


In [157]:
df_test['late_payment'] = df_test['Count_3-6_months_late'] + df_test['Count_6-12_months_late'] + df_test['Count_more_than_12_months_late']

In [158]:
df_test['cash_payment'] = df_test['perc_premium_paid_by_cash_credit'] * df_test['premium']

In [159]:
df_test['correct_payment'] = df_test['no_of_premiums_paid'] - df_test['late_payment']

In [160]:
cat = pd.get_dummies(df_test[['residence_area_type','sourcing_channel']])
df_test = pd.concat([df_test[df_test.columns.difference(['residence_area_type','sourcing_channel'])],cat[cat.columns.difference([
'residence_area_type_Rural','sourcing_channel_E'])]],axis = 1)

In [161]:
df_test = df_test[df_test.columns.difference(['renewal','incentives'])]

In [162]:
test_proba_cv = model_xgb.predict_proba(df_test)[:,1]
df_test['renewal'] = test_proba_cv

### Incentives set as 2% of Premium

In [163]:
df_test['incentives'] = 0.02 * df_test['premium']

In [164]:
df_test[['renewal','incentives']].to_csv('E:/Projects/AV_Hackathon/final_col.csv')